## implement a neural net

In [69]:
import numpy as np
import numpy.random as rd
from scipy.special import expit, logit

In [73]:
# this is a simple neural network with one hidden layer with n neurones
class Simple_nnet:

    def __init__(self, input_shape , hidden_layer_shape = 16, output_shape = 2) :
        self.input_shape = input_shape
        self.hidden_layer_shape = hidden_layer_shape
        self.output_shape = output_shape

        # init wheights and bias for hidden layer with random numbers 
        self.h_weights = rd.rand(hidden_layer_shape, input_shape)
        self.h_bias = rd.rand(hidden_layer_shape, 1)

        # init wheights and bias for output layer with random numbers 
        self.o_weights = rd.rand(hidden_layer_shape, input_shape)
        self.o_bias = rd.rand(hidden_layer_shape, 1)

    def process_input(self, input):
        # reshape input
        input = np.reshape(input, (input.shape[0], 1))
        # calculate hidden layer neurones
        a1 = (self.h_weights @ input )+ self.h_bias
        a1 = expit(a1) # apply sigmoid function

        # calculate output layer
        a2 = 
        # return a1 after applying sigmoid function
        return 

    def __str__(self):
        return str(self.input_shape) + " --> " + str(self.hidden_layer_shape) + " --> " + str(self.output_shape)



In [74]:
nnet1 = Simple_nnet(4, 8)

In [75]:
input1 = np.array([1,1,1,1])

In [76]:
nnet1.process_input(input1)

array([[0.94651834],
       [0.98152023],
       [0.83726249],
       [0.96379179],
       [0.89274987],
       [0.93493131],
       [0.89132526],
       [0.97023218]])

In [72]:
expit(a)

array([[0.9660009 ],
       [0.96336817],
       [0.91678827],
       [0.89746349],
       [0.85750068],
       [0.94202212],
       [0.94120056],
       [0.89968549]])